# 6.2 Стекинг

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.base import clone

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from matplotlib import pyplot as plt

%matplotlib inline

In [2]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz', sep=',', header=None)[:10000]

In [3]:
df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2


In [4]:
df.shape

(10000, 55)

In [5]:
features = list(range(0, 54))
target = 54

df = df[(df[target] == 1) | (df[target] == 2)]

In [6]:
cover_train, cover_test = train_test_split(df, test_size=0.5)

cover_X_train, cover_y_train = cover_train[features], cover_train[target]
cover_X_test, cover_y_test = cover_test[features], cover_test[target]

In [7]:
scaler = StandardScaler()
cover_X_train = scaler.fit_transform(cover_X_train)
cover_X_test = scaler.transform(cover_X_test)

Stacking — еще один способ объединить несколько алгоритмов в один, который часто используется как в решении реальных задач из промышленной сферы, так и в конкурсах на платформах вроде Kaggle.  
Подход использует понятие _базовых классификаторов_, каждый из которых независимо обучается на некотором (возможно одном и том же) множестве признаков, а также _мета-классификатора_, использующего предсказания базовых классификаторов как признаки.

Для избежания переобучения будем разбивать обучающую выборку на фолды.  
Например, фолды при разбиении на три части:  
``==*``  
``=*=``  
``*==``  

Это требуется для того, чтобы получить новые признаки (ответы алгоритмов на первом уровне) на всей обучающей выборке, т.е. ответы алгоритма на тех объектах, которые не были использованы во время обучения. В примере выше мы будем использовать ответы алгоритма, полученные на объектах звездочках. _Важно_: на каждом фолде мы обучаем алгоритм заново.

In [8]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    """
    Computes meta-features using the classifier.
    
    :arg clf: scikit-learn classifier
    :args X_train, y_train: training set
    :arg X_test: testing set
    :arg cv: cross-validation folding
    """
    X_meta_train = np.zeros_like(y_train, dtype=np.float32)
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)[:, 1]
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)[:, 1]
    
    return X_meta_train, X_meta_test

In [9]:
def generate_metafeatures(classifiers, X_train, X_test, y_train, cv):
    """
    Generates metafeatures using a list of classifiers.
    
    :arg classifiers: list of scikit-learn classifiers
    :args X_train, y_train: training set
    :arg X_test: testing set
    :arg cv: cross-validation folding
    """
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]
    
    stacked_features_train = np.vstack([
        features_train for features_train, features_test in features
    ]).T

    stacked_features_test = np.vstack([
        features_test for features_train, features_test in features
    ]).T
    
    return stacked_features_train, stacked_features_test

In [10]:
np.random.seed(42)

In [11]:
clf = GradientBoostingClassifier(n_estimators=300)
clf.fit(cover_X_train, cover_y_train)

accuracy_score(clf.predict(cover_X_test), cover_y_test)

0.7681465821000705

In [12]:
cv = KFold(n_splits=10, shuffle=True)

stacked_features_train, stacked_features_test = generate_metafeatures([
    LogisticRegression(C=0.001, penalty='l1', solver='liblinear', max_iter=5000),
    LogisticRegression(C=0.001, penalty='l2', solver='liblinear', max_iter=5000),  
    RandomForestClassifier(n_estimators=300, n_jobs=-1),
    GradientBoostingClassifier(n_estimators=300)
], cover_X_train, cover_X_test, cover_y_train.values, cv)

100%|██████████| 4/4 [00:20<00:00,  5.23s/it]


In [13]:
total_features_train = np.hstack([cover_X_train, stacked_features_train])
total_features_test = np.hstack([cover_X_test, stacked_features_test])

In [14]:
np.random.seed(42)
clf = LogisticRegression(penalty='none', solver='lbfgs')
clf.fit(stacked_features_train, cover_y_train)
accuracy_score(clf.predict(stacked_features_test), cover_y_test)

0.780831571529246

## Задание 6.1

Мы разобрали схему генерации признаков в стекинге, когда для тестовой выборки алгоритм заново переобучался на всей тренировочной выборке. Реализуйте схему, когда вместо этого производится агрегация ответов всех обученных на фолдах классификаторов на тестовой выборке при помощи усреднения.
Логика решения:

1. Создадим `X_meta_test`, заполним его нулями (по аналогии с `X_meta_train`);
2. Далее на каждом шаге, где мы обучаем `folded_clf.fit` (`X_fold_train`, `y_fold_train`) и его предсказания на `X_fold_predict` запихиваем в `X_meta_train[predict_fold_index]` добавим еще одну строку, где в `X_meta_test` будем добавлять предсказания вероятностей `folded_clf` на `X_test`. Их можно сразу складывать друг с другом или сохранить много массивов, тогда в конце их нужно будет все сложить, а потом делить на количество сплитов (количество массивов равно количеству сплитов в кросс - валидации);
3. После цикла останется только усреднить все эти массивы, это и будет наш `X_meta_test`.

За основу нужно взять следующий код:

```py
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    """    Эта функция подсчитывает признаки для мета-классификатора.     Они являются вероятностями классов при решении задачи многоклассовой классификации.    :arg clf: классификатор    :args X_train, y_train: обучающая выборка    :arg X_test: признаки тестовой выборки    :arg cv: класс, генерирующий фолды (KFold)    :returns X_meta_train, X_meta_test: новые признаки для обучающей и тестовой выборок    """
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(X_train), n_classes), dtype=np.float32)
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]

        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)

        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)

    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)

    X_meta_test = meta_clf.predict_proba(X_test)

    return X_meta_train, X_meta_test
```

In [15]:
def compute_meta_feature_mean(clf, X_train, X_test, y_train, cv):
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(X_train), n_classes), dtype=np.float32)
    X_meta_test  = np.zeros((len(X_test) , n_classes), dtype=np.float32)
    
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]

        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)

        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
        X_meta_test+= folded_clf.predict_proba(X_test)
    X_meta_test /= cv.n_splits
  
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)

    return X_meta_train, X_meta_test


In [18]:
clf = GradientBoostingClassifier(n_estimators=300)
cv = KFold(n_splits=10, shuffle=True)
compute_meta_feature_mean(clf, cover_X_train, cover_X_test, cover_y_train.values, cv)

(array([[0.11768798, 0.882312  ],
        [0.73624694, 0.26375306],
        [0.49182397, 0.508176  ],
        ...,
        [0.60861295, 0.39138702],
        [0.8461145 , 0.15388548],
        [0.14457297, 0.855427  ]], dtype=float32),
 array([[0.14526599, 0.85473401],
        [0.83996013, 0.16003987],
        [0.67311124, 0.32688876],
        ...,
        [0.02678432, 0.97321568],
        [0.00528169, 0.99471831],
        [0.98757941, 0.01242059]]))

Для следующих заданий используйте этот код:

In [19]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(y_train), n_classes), dtype=np.float32)

    splits = cv.split(X_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)
    
    return X_meta_train, X_meta_test

In [21]:
def generate_meta_features(classifiers, X_train, X_test, y_train, cv):
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]
    
    stacked_features_train = np.hstack([
        features_train for features_train, features_test in features
    ])

    stacked_features_test = np.hstack([
        features_test for features_train, features_test in features
    ])
    
    return stacked_features_train, stacked_features_test

In [23]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

def compute_metric(clf, X_train, y_train, X_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round(f1_score(y_test, y_test_pred, average='macro'), 6)

## Задание 6.2

Используйте функцию `generate_meta_features` для стекинга следующих алгоритмов:
- логистическая регрессия с L1-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — one-vs-rest, максимальное допустимое количество итераций — 2000
- логистическая регрессия с L2-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — multinomial, максимальное допустимое количество итераций — 2000
- случайный лес из 300 деревьев
- градиентный бустинг из 200 деревьев

Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.
Посчитайте качество при помощи передачи новых признаков в функцию *compute_metric*.

In [81]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.datasets import load_digits

In [37]:
dataset = load_digits()
X, y = dataset['data'], dataset['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [62]:
stack = [
    LogisticRegression(C=0.001, penalty='l1', solver='saga', max_iter=2000, multi_class='ovr'),
    LogisticRegression(C=0.001, penalty='l2', solver='saga', max_iter=2000, multi_class='multinomial'),  
    RandomForestClassifier(random_state=42, n_estimators=300, n_jobs=-1),
    GradientBoostingClassifier(random_state=42, n_estimators=200),
]
stacked_features_train, stacked_features_test = generate_meta_features(stack, X_train, X_test, y_train, cv)

100%|██████████| 4/4 [02:55<00:00, 43.91s/it]


In [67]:
clf = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000)

compute_metric(clf, X_train=stacked_features_train, y_train=y_train, X_test=stacked_features_test)

0.98339

## Задание 6.3

Используйте функцию `generate_meta_features` для стекинга следующих алгоритмов:
- случайный лес из 300 деревьев
- случайный лес из 200 экстремальных деревьев

Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — *auto* и солвером 'lbfgs'.

Посчитайте качество при помощи передачи новых признаков в функцию `compute_metric`.

In [71]:
stack = [
    RandomForestClassifier(random_state=42, n_estimators=300, n_jobs=-1),
    ExtraTreesClassifier(random_state=42, n_estimators=200, n_jobs=-1),
]
stacked_features_train, stacked_features_test = generate_meta_features(stack, X_train, X_test, y_train, cv)

100%|██████████| 2/2 [00:13<00:00,  6.65s/it]


In [72]:
clf = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000)

compute_metric(clf, X_train=stacked_features_train, y_train=y_train, X_test=stacked_features_test)

0.981296

## Задание 6.4

Используйте функцию `generate_meta_features` для стекинга следующих алгоритмов:

- метод ближайшего соседа (k-NN) со стандартными параметрами
- случайный лес из 300 экстремальных деревьев

Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — *auto* и солвером 'lbfgs'.

Посчитайте качество при помощи передачи новых признаков в функцию `compute_metric`.

In [76]:
stack = [
    KNeighborsClassifier(),
    ExtraTreesClassifier(random_state=42, n_estimators=300, n_jobs=-1),
]
stacked_features_train, stacked_features_test = generate_meta_features(stack, X_train, X_test, y_train, cv)

100%|██████████| 2/2 [00:06<00:00,  3.17s/it]


In [77]:
clf = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000)

compute_metric(clf, X_train=stacked_features_train, y_train=y_train, X_test=stacked_features_test)

0.987798

## Задание 6.5

Используйте функцию `generate_meta_features` для стекинга следующих алгоритмов:
- логистическая регрессия с L1-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — one-vs-rest, максимальное допустимоей количество итераций — 2000
- метод ближайшего соседа со стандартными параметрами
- случайный лес из 300 экстремальных деревьев
- AdaBoost со стандартными параметрами

Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — *auto* и солвером 'lbfgs'.

Посчитайте качество при помощи передачи новых признаков в функцию `compute_metric`.

In [79]:
stack = [
    LogisticRegression(C=0.001, penalty='l1', solver='saga', max_iter=2000, multi_class='ovr'),
    KNeighborsClassifier(),
    RandomForestClassifier(random_state=42, n_estimators=300, n_jobs=-1),
    AdaBoostClassifier(),
]
stacked_features_train, stacked_features_test = generate_meta_features(stack, X_train, X_test, y_train, cv)

100%|██████████| 4/4 [00:47<00:00, 11.78s/it]


In [80]:
clf = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000)

compute_metric(clf, X_train=stacked_features_train, y_train=y_train, X_test=stacked_features_test)

0.987798

## Задание 6.6

Используйте функцию `generate_meta_features` для стекинга следующих алгоритмов:
- случайный лес из 300 деревьев
- случайный лес из 300 экстремальных деревьев

Для генерации фолдов используйте класс `StratifiedKFold`, который позволяет делать так называемые стратифицированные разбиения (в каждом фолде будет одинаковое соотношение классов).
Для корректной работы необходимо подправить код в функции `compute_meta_feature`. Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — *auto* и солвером 'lbfgs'.

Посчитайте качество при помощи передачи новых признаков в функцию `compute_metric`. Количество фолдов = 10

In [85]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(y_train), n_classes), dtype=np.float32)

    splits = cv.split(X_train, y_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)
    
    return X_meta_train, X_meta_test

In [86]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [87]:
stack = [
    RandomForestClassifier(random_state=42, n_estimators=300, n_jobs=-1),
    ExtraTreesClassifier(random_state=42, n_estimators=300, n_jobs=-1),
]
stacked_features_train, stacked_features_test = generate_meta_features(stack, X_train, X_test, y_train, cv)


100%|██████████| 2/2 [00:15<00:00,  7.59s/it]


In [88]:
clf = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000)

compute_metric(clf, X_train=stacked_features_train, y_train=y_train, X_test=stacked_features_test)

0.981296

## Задание 6.7

В предыдущей задаче измените 10 фолдов на 20. Укажите полученное качество.

In [89]:
cv = StratifiedKFold(n_splits=20, shuffle=True, random_state=42)

In [90]:
stack = [
    RandomForestClassifier(random_state=42, n_estimators=300, n_jobs=-1),
    ExtraTreesClassifier(random_state=42, n_estimators=300, n_jobs=-1),
]
stacked_features_train, stacked_features_test = generate_meta_features(stack, X_train, X_test, y_train, cv)


100%|██████████| 2/2 [00:28<00:00, 14.05s/it]


In [91]:
clf = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000)

compute_metric(clf, X_train=stacked_features_train, y_train=y_train, X_test=stacked_features_test)

0.981296

## Задание 6.8

В предыдущей задаче укажите количество фолдов равным 5 и поменяйте мета-алгоритм на случайный лес со стандартными параметрами. Укажите полученное качество.

In [92]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [93]:
stack = [
    RandomForestClassifier(random_state=42, n_estimators=300, n_jobs=-1),
    ExtraTreesClassifier(random_state=42, n_estimators=300, n_jobs=-1),
]
stacked_features_train, stacked_features_test = generate_meta_features(stack, X_train, X_test, y_train, cv)


100%|██████████| 2/2 [00:07<00:00,  3.92s/it]


In [94]:
clf = RandomForestClassifier(random_state=42)

compute_metric(clf, X_train=stacked_features_train, y_train=y_train, X_test=stacked_features_test)

0.981661

## Задание 6.9

В предыдущей задаче поменяйте мета-алгоритм на метод ближайших соседей (k-NN) со стандартными параметрами. Укажите полученное качество.

In [95]:
clf = KNeighborsClassifier()

compute_metric(clf, X_train=stacked_features_train, y_train=y_train, X_test=stacked_features_test)

0.98417

## Задание 6.10

В предыдущей задаче поменяйте мета-алгоритм на градиентный бустинг со стандартными параметрами. Укажите полученное качество.

In [96]:
clf = GradientBoostingClassifier()

compute_metric(clf, X_train=stacked_features_train, y_train=y_train, X_test=stacked_features_test)

0.984925

## Задание 6.11

Используйте функцию `generate_meta_features` для стекинга следующих алгоритмов:
- случайный лес из 300 деревьев, критерий Джини, максимальная глубина — 24
- случайный лес из 300 экстремальных деревьев

Для генерации фолдов используйте класс *StratifiedKFold*, который позволяет делать так называемые стратифицированные разбиения (в каждом фолде будет одинаковое соотношение классов).

Для генерации фолдов используйте класс *StratifiedKFold* и поправленный Вами ранее код в функции `compute_meta_feature`.

Выполните разбиение на 3 фолда.

Как мета-алгортм используйте случайный лес из 100 экстремальных деревьев. Посчитайте качество при помощи передачи новых признаков в функцию `compute_metric`.

In [97]:
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

In [99]:
stack = [
    RandomForestClassifier(random_state=42, criterion='gini', max_depth=24, n_estimators=300, n_jobs=-1),
    ExtraTreesClassifier(random_state=42, n_jobs=-1),
]
stacked_features_train, stacked_features_test = generate_meta_features(stack, X_train, X_test, y_train, cv)


100%|██████████| 2/2 [00:03<00:00,  1.85s/it]


In [101]:
clf = ExtraTreesClassifier(n_estimators=100, random_state=42)

compute_metric(clf, X_train=stacked_features_train, y_train=y_train, X_test=stacked_features_test)

0.981669

## Задание 6.12

Обучите на тренировочной выборке следующие алгоритмы:
- случайный лес из 300 деревьев, критерий Джини, максимальная глубина — 24
- случайный лес из 300 экстремальных деревьев
- логистическую регрессию со стандартными параметрами

Усредните их ответы на тестовой выборке и посчитайте качество аналогично функции compute_metric (F1-score с макро-усреднением, округленный до 6 знака).

In [130]:
solver1 = RandomForestClassifier(random_state=42, criterion='gini', max_depth=24, n_estimators=300, n_jobs=-1)
solver1.fit(X_train, y_train)
y_pred1 = solver1.predict(X_test)

In [131]:
solver2 = ExtraTreesClassifier(random_state=42, n_estimators=300, n_jobs=-1)
solver2.fit(X_train, y_train)
y_pred2 = solver2.predict(X_test)

In [132]:
solver3 = LogisticRegression()
solver3.fit(X_train, y_train)
y_pred3 = solver3.predict(X_test)

/data/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [134]:
y_pred = np.floor((y_pred1 + y_pred2 + y_pred3) / 3)

In [135]:
score = f1_score(y_test, y_pred, average='macro')

np.round(score, 6)

0.965569